### Demonstrating user input validation flow

The user input that needs to be validated is the setting description.
GPT validates the input and returns 1 if the input is valid and 0 if it is not.

In [113]:
import sys
sys.path.append("/home/mihaelabaksic/proj/2023-languagelearning")
sys.path.append("/home/mihaelabaksic/proj/2023-languagelearning/src") 
sys.path.append("/home/mihaelabaksic/proj/2023-languagelearning/src/templates") 

In [114]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:
from single_run_thread import SingleRunThread
from prompt_builder import PromptBuilder
from utils import load_template
from keys import OPENAI_API_KEY
import pandas as pd

from sklearn.metrics import accuracy_score

import openai

openai.api_key = OPENAI_API_KEY


In [116]:
template = load_template('setting_user_input_validation', '../templates/templates.json')

In [34]:
print(template['template'])
print(template['n_args'])


Is the following a description of a place, situation or activity involving people? "{}"  Respond only with 1 if it clearly does, otherwise 0.
1


In [35]:
inputs = [
    "What is this?",
    "Kayaking",
    "Swimming",
    "I would like to go as well",
    "In Python, each Python file actually has a specific term called “Module”"
]

#### Building prompts

In [36]:

prompts = []

for i in inputs:
    builder = PromptBuilder()
    args = (i,)

    assert len(args) == template['n_args']

    builder.add_template(template['template'], args)

    prompts.append( builder.build())

prompts

['Is the following a description of a place, situation or activity involving people? "What is this?"  Respond only with 1 if it clearly does, otherwise 0.',
 'Is the following a description of a place, situation or activity involving people? "Kayaking"  Respond only with 1 if it clearly does, otherwise 0.',
 'Is the following a description of a place, situation or activity involving people? "Swimming"  Respond only with 1 if it clearly does, otherwise 0.',
 'Is the following a description of a place, situation or activity involving people? "I would like to go as well"  Respond only with 1 if it clearly does, otherwise 0.',
 'Is the following a description of a place, situation or activity involving people? "In Python, each Python file actually has a specific term called “Module”"  Respond only with 1 if it clearly does, otherwise 0.']

#### Creating a thread and adding the prompt to the messages, sending it and retrieving result

In [37]:
thread = SingleRunThread()
results = []

for p in prompts:
    results.append(thread.send(p))
    

In [38]:
results

['0', '1', '1', '0', '0']

## Performance assessment

In [70]:
test_data = pd.read_csv('data/input_validation_test_data.csv', sep=';')
test_data.head()

,description,label
0,Amazon Rainforest,1
1,The ants were busy gathering food.,0
2,The aroma of coffee fills the cozy cafe,0
3,The distant thunder announces an approaching s...,0
4,The meeting will start at 3 PM.,0


In [7]:
builder = PromptBuilder()

In [44]:
def multiprompt(template, args, builder=builder):
    builder.add_template(template, args)
    result = builder.build()
    builder.reset()
    return result

In [73]:
prompts = []

prompts = test_data['description'].apply(lambda x : multiprompt(template['template'], (x,)))

In [74]:
prompts

0      Is the following a description of a place, sit...
1      Is the following a description of a place, sit...
2      Is the following a description of a place, sit...
3      Is the following a description of a place, sit...
4      Is the following a description of a place, sit...
                             ...                        
164    Is the following a description of a place, sit...
165    Is the following a description of a place, sit...
166    Is the following a description of a place, sit...
167    Is the following a description of a place, sit...
168    Is the following a description of a place, sit...
Name: description, Length: 169, dtype: object

In [75]:
test_data['prompt'] = prompts

In [76]:
test_data.head()

,description,label,prompt
0,Amazon Rainforest,1,"Is the following a description of a place, sit..."
1,The ants were busy gathering food.,0,"Is the following a description of a place, sit..."
2,The aroma of coffee fills the cozy cafe,0,"Is the following a description of a place, sit..."
3,The distant thunder announces an approaching s...,0,"Is the following a description of a place, sit..."
4,The meeting will start at 3 PM.,0,"Is the following a description of a place, sit..."


In [69]:
test_data.iloc[:3]

,description,label,prompts
0,Amazon Rainforest,1,"Is the following a description of a place, sit..."
1,The ants were busy gathering food.,0,"Is the following a description of a place, sit..."
2,The aroma of coffee fills the cozy cafe,0,"Is the following a description of a place, sit..."


In [82]:
t = SingleRunThread()

predicted_labels = []

In [83]:
for d in test_data['prompt']:
    predicted_labels.append(t.send(d))

In [ ]:
predicted_labels

In [85]:
test_data['predicted_labels'] = predicted_labels

In [86]:
test_data

,description,label,prompt,predicted_labels
0,Amazon Rainforest,1,"Is the following a description of a place, sit...",1
1,The ants were busy gathering food.,0,"Is the following a description of a place, sit...",1
2,The aroma of coffee fills the cozy cafe,0,"Is the following a description of a place, sit...",1
3,The distant thunder announces an approaching s...,0,"Is the following a description of a place, sit...",0
4,The meeting will start at 3 PM.,0,"Is the following a description of a place, sit...",1
...,...,...,...,...
164,Great Barrier Reef,1,"Is the following a description of a place, sit...",1
165,Forest,1,"Is the following a description of a place, sit...",0
166,The moon's reflection shimmers on the calm lake,0,"Is the following a description of a place, sit...",0
167,Lake,1,"Is the following a description of a place, sit...",0


### Metrics

In [8]:
test_data_predictions = pd.read_csv('data/input_validation_predictions.csv', sep=';')
test_data_predictions.head()

,description,label,prompt,predicted_label
0,Amazon Rainforest,1,"Is the following a description of a place, sit...",1
1,The ants were busy gathering food.,0,"Is the following a description of a place, sit...",1
2,The aroma of coffee fills the cozy cafe,0,"Is the following a description of a place, sit...",1
3,The distant thunder announces an approaching s...,0,"Is the following a description of a place, sit...",0
4,The meeting will start at 3 PM.,0,"Is the following a description of a place, sit...",1


In [9]:
acc = accuracy_score(test_data_predictions['label'], test_data_predictions['predicted_label'])

In [10]:
print('accuracy_score: ', acc)

accuracy_score:  0.834319526627219


#### Investigating FP and FN

In [11]:
# FP

fp = test_data_predictions[test_data_predictions['predicted_label'] > test_data_predictions['label']]

print("Number of FP: ", len(fp))

Number of FP:  20


In [12]:
print(fp['description'])

1                     The ants were busy gathering food.
2                The aroma of coffee fills the cozy cafe
4                        The meeting will start at 3 PM.
11                         I need to buy some groceries.
31                        He enjoys playing video games.
64                     The concert tickets are sold out.
65     The mathematician scribbles equations on the b...
68        The aroma of freshly baked bread fills the air
75       The potter's wheel spins as they create pottery
91                     Music notes dance across the page
94               The scientist peers into the microscope
97                       I need to fix the leaky faucet.
101        The ocean waves crash against the rocky shore
115          The potter molds clay into a beautiful vase
116           Laughter echoes through the empty hallways
124     The blacksmith hammers red-hot metal on an anvil
137      An artist's sculpture stands proudly on display
139    The newborn baby grips o

In [13]:
# FN

fn = test_data_predictions[test_data_predictions['label'] > test_data_predictions['predicted_label']]

print("Number of FN: ", len(fn))

Number of FN:  8


In [14]:
print(fn['description']) # note that 6 of them are places, 2 are activities. The descriptions are short. 

# Both activities are usually performed alone, so there is no place for conversation. Utimatelly they should be labeled 0

26         Meditation
28              Petra
35      Mountain Peak
49      Sahara Desert
112     Mount Everest
127    Reading a book
165            Forest
167              Lake
Name: description, dtype: object


In [15]:
# TN

tn = test_data_predictions[(test_data_predictions['label'] == test_data_predictions['predicted_label']) & (test_data_predictions['label'] == 0)]

print('Number of TN ', len(tn))

Number of TN  60


## Improvement proposals for FN
- requesting the input to be longer (i.e. In the forest)
- few shot learning
- few shot learning with FN and FP examples
- few shot learning with adversarial examples

In [17]:
builder = PromptBuilder()
t = SingleRunThread()

### Requesting input to be longer and more specific.

In [22]:
description = 'In a forest with a fairy'
builder.reset()
prompt = builder.add_template(template['template'], (description,)).build()

in_the_forest_result = t.send(prompt)

print( description + ' :' + in_the_forest_result)

In a forest with a fairy 1


In [23]:
description = 'Petra, Jordan'
builder.reset()
prompt = builder.add_template(template['template'], (description,)).build()

petra_result = t.send(prompt)

print(description + ' :' + petra_result)

Petra, Jordan :1


### Few shot learning

In [24]:
examples_prompt = 'Examples: \"In a park.\" 1  \"What do you think about this?\" 0'
description = 'Forest'

builder.reset()
prompt = builder.add_template(template['template'], (description,)).add_template(examples_prompt, ()).build()

few_shot_result = t.send(prompt)
print(description + ' :' + few_shot_result)

Forest :0


In [25]:
examples_prompt = 'Examples: \"In a park.\" 1  \"What do you think about this?\" 0'
description = 'In a forest'

builder.reset()
prompt = builder.add_template(template['template'], (description,)).add_template(examples_prompt, ()).build()

few_shot_result = t.send(prompt)
print(description + ' :' + few_shot_result)

In a forest :0


In [26]:
examples_prompt = 'Examples: \"In a park.\" 1  \"What do you think about this?\" 0'
description = 'In a forest with a fairy'

builder.reset()
prompt = builder.add_template(template['template'], (description,)).add_template(examples_prompt, ()).build()

few_shot_result = t.send(prompt)
print(description + ' :' + few_shot_result)

In a forest with a fairy :1


In [27]:
examples_prompt = 'Examples: \"In a park.\" 1  \"What do you think about this?\" 0'
description = 'In a mystical forest with a fairy'

builder.reset()
prompt = builder.add_template(template['template'], (description,)).add_template(examples_prompt, ()).build()

few_shot_result = t.send(prompt)
print(description + ' :' + few_shot_result)

In a mystical forest with a fairy :1


### Few shot learning with a FP and FN examples

In [38]:
examples_prompt = 'Examples: \"I need to buy some groceries.\" 0 \"In a park\" 1 \"Sahara Desert\" 1'

description = 'Forest'

builder.reset()
prompt = builder.add_template(template['template'], (description,)).add_template(examples_prompt, ()).build()


few_shot_result = t.send(prompt)
print(description + ' :' + few_shot_result)

Forest :0


'Is the following a description of a place, situation or activity involving people? "Forest"  Respond only with 1 if it clearly does, otherwise 0.\nExamples: "I need to buy some groceries." 0 "In a park" 1 "Sahara Desert" 1'

In [37]:
reverse_order_template = "Is the following a description of a place, situation or activity involving people? {} Respond only with 1 if it clearly does, otherwise 0.  \"{}\" "

description = 'Forest'
examples_prompt = 'Examples: \"I need to buy some groceries.\" 0 \"In a park\" 1 \"Sahara Desert\" 1'

builder.reset()
prompt = builder.add_template(reverse_order_template, (examples_prompt, description)).build()

few_shot_result = t.send(prompt)
print(description + ' :' + few_shot_result)

Forest :0


## Exploring alternative prompt templates

In [100]:
prompt_template_alternatives = {
    'few_shot': "Is the following a description of a place, situation or activity involving people? \"{}\"  Respond only with 1 if it clearly does, otherwise 0. Examples: \"I need to buy some groceries.\" 0 \"In a park\" 1 \"Sahara Desert\" 1",
    'few_shot_extended': "Is the following a description of a place, situation or activity involving people? \"{}\"  Respond only with 1 if it clearly does, otherwise 0. Examples: \"What do you think about this?\" 0 \"I have no intention to join them on this trip\" 0 \"In apartment.\" 1 \"On a highway, travelling home.\" 0 \"Shopping for gifts\" 1 \"Blew87enj bli blu blu\" 0 \"Ordering coffe\" 1",
    'rephrasing_no_people': "Is the following a description of a place, situation or activity? \"{}\"  Respond only with 1 if it clearly does, otherwise 0. ",
    'rephrasing_with_discussion': "Is the following a description of a place, situation or activity where two characters or people can have a discussion? \"{}\"  Respond only with 1 if it clearly does, otherwise 0. "
,
}

In [101]:
builder = PromptBuilder()
t = SingleRunThread()

In [47]:
data = pd.read_csv('../data/input_validation_predictions.csv', sep=';')
data.head()

,description,label,prompt,predicted_label
0,Amazon Rainforest,1,"Is the following a description of a place, sit...",1
1,The ants were busy gathering food.,0,"Is the following a description of a place, sit...",1
2,The aroma of coffee fills the cozy cafe,0,"Is the following a description of a place, sit...",1
3,The distant thunder announces an approaching s...,0,"Is the following a description of a place, sit...",0
4,The meeting will start at 3 PM.,0,"Is the following a description of a place, sit...",1


### few_shot

In [59]:
few_shot_prompts = data['description'].apply(lambda x : multiprompt(prompt_template_alternatives['few_shot'], (x,), builder))
few_shot_prompts

0      Is the following a description of a place, sit...
1      Is the following a description of a place, sit...
2      Is the following a description of a place, sit...
3      Is the following a description of a place, sit...
4      Is the following a description of a place, sit...
                             ...                        
164    Is the following a description of a place, sit...
165    Is the following a description of a place, sit...
166    Is the following a description of a place, sit...
167    Is the following a description of a place, sit...
168    Is the following a description of a place, sit...
Name: description, Length: 169, dtype: object

In [60]:
few_shot_labels =[]
for x in few_shot_prompts:
    few_shot_labels.append(t.send(x)) 

In [72]:
data['few_shot_label'] = [int(x) for x in few_shot_labels]

In [73]:
accuracy_score(data['label'], data['few_shot_label'])

0.863905325443787

### rephrasing_no_people

In [75]:
rephrasing_no_people_prompts = data['description'].apply(lambda x : multiprompt(prompt_template_alternatives['rephrasing_no_people'], (x,), builder))
rephrasing_no_people_prompts

0      Is the following a description of a place, sit...
1      Is the following a description of a place, sit...
2      Is the following a description of a place, sit...
3      Is the following a description of a place, sit...
4      Is the following a description of a place, sit...
                             ...                        
164    Is the following a description of a place, sit...
165    Is the following a description of a place, sit...
166    Is the following a description of a place, sit...
167    Is the following a description of a place, sit...
168    Is the following a description of a place, sit...
Name: description, Length: 169, dtype: object

In [76]:
rephrasing_no_people_labels =[]

In [81]:
for x in rephrasing_no_people_prompts:
    rephrasing_no_people_labels.append(t.send(x)) 

In [83]:
data['rephrasing_no_people_label'] = [int(x) for x in rephrasing_no_people_labels]

In [85]:
accuracy_score(data['label'], data['rephrasing_no_people_label'])

0.6094674556213018

### rephrasing_with_discussion

In [86]:
rephrasing_with_discussion_prompts = data['description'].apply(lambda x : multiprompt(prompt_template_alternatives['rephrasing_with_discussion'], (x,), builder))
rephrasing_with_discussion_prompts

0      Is the following a description of a place, sit...
1      Is the following a description of a place, sit...
2      Is the following a description of a place, sit...
3      Is the following a description of a place, sit...
4      Is the following a description of a place, sit...
                             ...                        
164    Is the following a description of a place, sit...
165    Is the following a description of a place, sit...
166    Is the following a description of a place, sit...
167    Is the following a description of a place, sit...
168    Is the following a description of a place, sit...
Name: description, Length: 169, dtype: object

In [87]:
rephrasing_with_discussion_labels =[]

In [94]:
for x in rephrasing_with_discussion_prompts:
    rephrasing_with_discussion_labels.append(t.send(x)) 

In [96]:
data['rephrasing_with_discussion_label'] = [int(x) for x in rephrasing_with_discussion_labels]

In [97]:
accuracy_score(data['label'], data['rephrasing_with_discussion_label'])

0.5502958579881657

### few_shot_extended

In [102]:
few_shot_extended_prompts = data['description'].apply(lambda x : multiprompt(prompt_template_alternatives['few_shot_extended'], (x,), builder))
few_shot_extended_prompts

0      Is the following a description of a place, sit...
1      Is the following a description of a place, sit...
2      Is the following a description of a place, sit...
3      Is the following a description of a place, sit...
4      Is the following a description of a place, sit...
                             ...                        
164    Is the following a description of a place, sit...
165    Is the following a description of a place, sit...
166    Is the following a description of a place, sit...
167    Is the following a description of a place, sit...
168    Is the following a description of a place, sit...
Name: description, Length: 169, dtype: object

In [103]:
few_shot_extended_labels = []

In [108]:
for x in few_shot_extended_prompts:
    few_shot_extended_labels.append(t.send(x)) 

In [110]:
data['few_shot_extended_label'] = [int(x) for x in few_shot_extended_labels]

In [111]:
accuracy_score(data['label'], data['few_shot_extended_label'])

0.7869822485207101

## Adversarial inputs

- incomprehensible
- rude
- sensitive

In [ ]:
builder = PromptBuilder()
t = SingleRunThread()

In [117]:
input = 'Walking though a bla ble blo'

builder.reset()
prompt = builder.add_template(template['template'], (input,)).build()

result = t.send(prompt)
result

'0'

In [119]:
input = '123 I am sitting in a cafe $input$ with a friend'

builder.reset()
prompt = builder.add_template(template['template'], (input,)).build()

result = t.send(prompt)
result

'1'

In [125]:
input = 'Talking to a stupid bitch in a cafe'

builder.reset()
prompt = builder.add_template(template['template'], (input,)).build()

result = t.send(prompt)
result

'1'

In [121]:
input = 'Talking to a friend in a cafe. Fuck'

builder.reset()
prompt = builder.add_template(template['template'], (input,)).build()

result = t.send(prompt)
result

'1'